# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

<div class="alert alert-block alert-warning">
<b>Note:</b>
As of v0.5, most outputs, including availability, return values as a pandas DataFrame, regardless of the aggregation level.
</div>


In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
print(f"Project total: {total * 100:.1f}%")

Project total: 96.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.96,0.98,0.98,0.96,0.89,...,0.98,0.97,0.98,0.86,0.98


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.97
2004,0.96
2005,0.96
2006,0.96
2007,0.98
2008,0.95
2009,0.97
2010,0.96
2011,0.95


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.96
3,0.96
4,0.96
5,0.96
6,0.96
7,0.97
8,0.96
9,0.96


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.99
     2          0.99
     3          0.98
     4          0.97
     5          0.96
...              ...
2012 8          0.97
     9          0.95
     10         0.94
     11         0.94
     12         0.97

[120 rows x 1 columns]

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.46,0.46,0.47,0.46,0.42,...,0.47,0.46,0.47,0.41,0.47


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.47
2006,0.45
2007,0.50
2008,0.49
2009,0.46
2010,0.41
2011,0.48


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.50
3,0.50
4,0.41
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.63
     2          0.37
     3          0.37
     4          0.52
     5          0.34
...              ...
2012 8          0.31
     9          0.54
     10         0.51
     11         0.51
     12         0.61

[120 rows x 1 columns]

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.98
2004,0.95
2005,1.05
2006,0.99
2007,1.01
2008,1.00
2009,0.98
2010,1.02
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,1.01
2,1.01
3,1.01
4,1.02
5,1.01
6,1.03
7,0.99
8,1.00
9,0.98


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year")

Completion Rate
year month                 
2003 1                 0.90
     2                 1.10
     3                 0.99
     4                 0.96
     5                 1.00
...                     ...
2012 8                 0.94
     9                 1.01
     10                0.77
     11                1.22
     12                1.10

[120 rows x 1 columns]

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $424,671.71/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","2,071,011.77","80,670,416.67"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"7,129,693.15"
2004,"8,208,688.99"
2005,"8,650,035.71"
2006,"15,940,144.58"
2007,"4,029,007.50"
2008,"18,133,722.18"
2009,"7,211,316.09"
2010,"7,039,971.43"
2011,"12,712,063.81"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,083.33","543,010.42","543,010.42","237,500.00","5,510,937.50"
2,"496,052.08","495,250.00","496,125.00",0.00,0.00
3,"541,989.58","542,864.58","541,989.58","75,442.64","8,534,180.56"
4,"524,635.42","524,635.42","524,635.42","102,155.57","8,912,822.92"
5,"542,500.00","542,500.00","542,500.00","81,934.97","8,857,291.67"
6,"525,000.00","525,000.00","525,000.00","90,214.59","4,897,048.61"
7,"542,500.00","542,500.00","542,500.00","18,939.20","6,276,696.18"
8,"542,500.00","542,500.00","542,500.00","201,938.99","4,650,000.00"
9,"525,000.00","525,000.00","525,000.00","427,897.81","2,231,802.08"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
...                            ...                     ...   
2012 8                   54,250.00               54,250.00   
     9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00                  0.00               0.00  
...                            ...                   ...                ...  
2012 8                   54,250.00                  0.00               0.00  
     9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00       3,051,545.14  
     12                  54,250.00             72,003.73       2,706,250.00  

[120 rows x 5 columns]

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.91,0.96


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.83,0.93
2004,1.00,1.00,1.00,0.93,0.97
2005,1.00,1.00,1.00,0.86,0.93
2006,1.00,1.00,1.00,0.91,0.97
2007,1.00,1.00,1.00,0.95,1.00
2008,1.00,1.00,1.00,0.93,0.97
2009,1.00,1.00,1.00,0.94,0.94
2010,1.00,1.00,1.00,0.83,0.93
2011,1.00,1.00,1.00,0.85,0.96


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"124,220.42"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 3,Crew Transfer Vessel 2,Crew Transfer Vessel 1,Heavy Lift Vessel,Field Support Vessel
0,2003,"8,794.84","1,915.75","1,894.75","1,963.50","1,916.84","1,104.00"
1,2004,"14,385.50","3,896.50","3,984.50","3,909.50","1,959.00",636.00
2,2005,"15,641.47","3,307.25","3,333.75","3,514.75","3,341.72","2,144.00"
3,2006,"14,236.66","3,031.75","2,888.25","2,931.50","4,254.66","1,130.50"
4,2007,"10,139.00","3,114.50","3,092.25","2,950.25",232.50,749.50
5,2008,"15,063.72","2,713.75","2,758.75","2,796.50","5,658.72","1,136.00"
6,2009,"9,782.00","2,563.75","2,499.50","2,463.00","1,642.75",613.00
7,2010,"10,698.82","2,437.50","2,703.25","2,623.75","1,830.32","1,104.00"
8,2011,"13,289.41","2,655.00","2,246.75","2,339.75","3,839.91","2,208.00"
9,2012,"12,188.99","2,582.25","2,630.75","2,727.75","3,648.24",600.00


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
...                      ...
2012 8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

[120 rows x 1 columns]

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,248,476.56"
Repair,"21,693,667.53"
Crew Transfer,"1,153,953.13"
Site Travel,0.00
Mobilization,"7,500,000.00"
Weather Delay,"32,309,544.27"
No Requests,"15,427,212.36"
Not in Shift,"21,588,355.83"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,248,476.56","2,248,476.56"
Repair,0.00,0.00,0.00,"21,693,667.53","21,693,667.53"
Crew Transfer,0.00,0.00,0.00,"1,153,953.13","1,153,953.13"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"7,500,000.00","7,500,000.00"
Weather Delay,0.00,0.00,0.00,"32,309,544.27","32,309,544.27"
No Requests,0.00,0.00,0.00,"15,427,212.36","15,427,212.36"
Not in Shift,0.00,0.00,0.00,"21,588,355.83","21,588,355.83"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False)

total_cost
year reason                    
2003 Maintenance      61,250.00
     Repair        1,928,382.81
     Crew Transfer    90,635.42
     Site Travel           0.00
     Mobilization    500,000.00
...                         ...
2012 Site Travel           0.00
     Mobilization  1,000,000.00
     Weather Delay 3,419,748.26
     No Requests   3,114,677.08
     Not in Shift  2,201,220.75

[80 rows x 1 columns]

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False)

total_cost
month reason                    
1     Maintenance     178,809.90
      Repair        1,350,700.52
      Crew Transfer    90,450.52
      Site Travel           0.00
      Mobilization          0.00
...                          ...
12    Site Travel           0.00
      Mobilization  1,000,000.00
      Weather Delay 4,236,050.35
      No Requests   1,466,466.15
      Not in Shift  2,434,269.28

[96 rows x 1 columns]

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False)

total_cost
year month reason                    
2003 1     Maintenance           0.00
           Repair           18,921.88
           Crew Transfer     2,479.17
           Site Travel           0.00
           Mobilization          0.00
...                               ...
2012 12    Site Travel           0.00
           Mobilization          0.00
           Weather Delay 1,135,135.42
           No Requests      71,330.73
           Not in Shift    798,579.25

[960 rows x 1 columns]

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"116,998,260.37"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   53,744,663.14   
          maintenance            0.00              0.00    2,248,476.56   
          repair                 0.00              0.00   21,693,667.53   

                         total_cost  
component action                     
turbine   delay       53,744,663.14  
          maintenance  2,248,476.56  
          repair      21,693,667.53

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"2,688,500.00",0.00,"5,361,481.77","8,049,981.77"
2004,turbine,"4,641,000.00",0.00,"7,356,989.58","11,997,989.58"
2005,turbine,"3,757,500.00",0.00,"5,915,346.36","9,672,846.36"
2006,turbine,"5,617,500.00",0.00,"14,015,302.95","19,632,802.95"
2007,turbine,"2,606,000.00",0.00,"3,034,142.92","5,640,142.92"
2008,turbine,"5,387,000.00",0.00,"15,162,910.59","20,549,910.59"
2009,turbine,"2,409,000.00",0.00,"3,819,481.77","6,228,481.77"
2010,turbine,"2,636,000.00",0.00,"4,759,809.90","7,395,809.90"
2011,turbine,"3,993,500.00",0.00,"10,671,159.11","14,664,659.11"


In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"3,013,500.00",0.00,"6,670,460.94","9,683,960.94"
2,turbine,"1,249,000.00",0.00,"979,726.56","2,228,726.56"
3,turbine,"3,200,500.00",0.00,"8,134,412.33","11,334,912.33"
4,turbine,"3,958,500.00",0.00,"6,512,869.79","10,471,369.79"
5,turbine,"4,726,000.00",0.00,"8,506,492.19","13,232,492.19"
6,turbine,"2,858,000.00",0.00,"4,651,415.80","7,509,415.80"
7,turbine,"2,922,000.00",0.00,"4,835,316.84","7,757,316.84"
8,turbine,"2,516,500.00",0.00,"5,632,505.21","8,149,005.21"
9,turbine,"2,428,000.00",0.00,"2,782,968.44","5,210,968.44"


In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         31,500.00              0.00       49,875.00   
     2     turbine         22,000.00              0.00       43,859.38   
     3     turbine         31,500.00              0.00       31,864.58   
     4     turbine         61,000.00              0.00       45,408.85   
     5     turbine         42,500.00              0.00       43,786.46   
     6     turbine        442,000.00              0.00    1,213,250.00   
     7     turbine      1,390,000.00              0.00    3,483,531.25   
     8     turbine         15,000.00              0.00       40,614.58   
     9     turbine         60,000.00              0.00       52,700.52   
     10    turbine         97,000.00              0.00       93,078.13   
     11    turbine        316,500.00              0.00      144,184.90   
     12    turbine        179,500.00              0.00      119,328.13   
2004 1     turbine        148,500.00              0.00      162,750.00   
     2     turbine        196,000.00              0.00      151,375.00   
     3     turbine        254,500.00              0.00      162,750.00   
     4     turbine        581,500.00              0.00    1,661,447.92   
     5     turbine      1,933,000.00              0.00    4,174,052.08   
     6     turbine        200,000.00              0.00      154,000.00   
     7     turbine        183,500.00              0.00       76,908.85   
     8     turbine        271,000.00              0.00      203,776.04   
     9     turbine        255,000.00              0.00      135,898.44   
     10    turbine        244,000.00              0.00      156,406.25   
     11    turbine        170,000.00              0.00      155,750.00   
     12    turbine        204,000.00              0.00      161,875.00   
2005 1     turbine        149,500.00              0.00      158,375.00   
     2     turbine        130,000.00              0.00      146,125.00   
     3     turbine        404,000.00              0.00      232,559.90   
     4     turbine        332,000.00              0.00      141,822.92   

                       total_cost  
year month component               
2003 1     turbine      81,375.00  
     2     turbine      65,859.38  
     3     turbine      63,364.58  
     4     turbine     106,408.85  
     5     turbine      86,286.46  
     6     turbine   1,655,250.00  
     7     turbine   4,873,531.25  
     8     turbine      55,614.58  
     9     turbine     112,700.52  
     10    turbine     190,078.13  
     11    turbine     460,684.90  
     12    turbine     298,828.13  
2004 1     turbine     311,250.00  
     2     turbine     347,375.00  
     3     turbine     417,250.00  
     4     turbine   2,242,947.92  
     5     turbine   6,107,052.08  
     6     turbine     354,000.00  
     7     turbine     260,408.85  
     8     turbine     474,776.04  
     9     turbine     390,898.44  
     10    turbine     400,406.25  
     11    turbine     325,750.00  
     12    turbine     365,875.00  
2005 1     turbine     307,875.00  
     2     turbine     276,125.00  
     3     turbine     636,559.90  
     4     turbine     473,822.92

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,...,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low")

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
...                ...
2012 8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09

[120 rows x 1 columns]

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"156,091,940.44"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"11,418,201.15"
2004,"14,454,080.58"
2005,"14,007,543.71"
2006,"23,157,652.58"
2007,"8,235,015.50"
2008,"25,125,113.77"
2009,"11,220,324.09"
2010,"11,275,979.43"
2011,"18,305,571.81"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"11,749,952.57"
2,"3,976,981.23"
3,"14,795,877.86"
4,"15,862,459.80"
5,"16,651,637.54"
6,"10,735,338.28"
7,"12,204,046.28"
8,"10,354,849.90"
9,"7,977,774.97"


In [52]:
metrics.opex("month-year")

OpEx
year month             
2003 1       331,672.34
     2       291,740.34
     3       330,141.09
     4       350,007.51
     5       341,141.09
...                 ...
2012 8       334,141.09
     9       418,007.51
     10      391,641.09
     11    4,229,052.65
     12    4,433,894.82

[120 rows x 1 columns]

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
...               ...
2012 8              0
     9              0
     10             0
     11             0
     12             0

[120 rows x 1 columns]

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"220,656.43","75,297.25","67,896.50",516
major repair,"95,876.73","2,675.16","1,976.36",39
major replacement,"168,963.76","10,650.15","168,963.76",59
manual reset,"223,246.85","37,004.00","13,368.25",5699
medium repair,"12,968.31","9,887.50","7,701.00",204
minor repair,"72,166.36","31,663.50","15,402.00",2254


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,643,078,723.00"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,...,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,643,078,723.00","121,213,533.00","122,036,827.00","123,941,577.50","121,223,575.50",...,"123,277,011.00","121,511,454.50","122,486,948.50","108,064,256.50","122,658,093.50"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"858,010,100.50"
2004,"936,412,901.00"
2005,"985,389,279.50"
2006,"946,055,672.00"
2007,"1,048,318,566.00"
2008,"1,041,391,821.00"
2009,"963,656,315.50"
2010,"858,149,684.50"
2011,"1,012,508,298.50"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,077,510,099.00"
2,"818,625,358.00"
3,"897,235,655.50"
4,"709,982,068.00"
5,"690,436,494.50"
6,"558,496,037.00"
7,"591,497,220.00"
8,"643,701,164.00"
9,"791,234,311.50"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False)

windfarm
year month               
2003 1     112,126,571.00
     2      59,253,140.00
     3      66,812,997.50
     4      90,273,133.50
     5      60,187,287.00
...                   ...
2012 8      55,250,830.50
     9      92,926,933.50
     10     90,835,863.50
     11     88,117,262.50
     12    109,338,081.50

[120 rows x 1 columns]

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"196,895,685.05"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"11,418,201.15"
2004,"14,454,080.58"
2005,"14,007,543.71"
2006,"23,157,652.58"
2007,"8,235,015.50"
2008,"25,125,113.77"
2009,"11,220,324.09"
2010,"11,275,979.43"
2011,"18,305,571.81"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"11,749,952.57"
2,"3,976,981.23"
3,"14,795,877.86"
4,"15,862,459.80"
5,"16,651,637.54"
6,"10,735,338.28"
7,"12,204,046.28"
8,"10,354,849.90"
9,"7,977,774.97"


In [63]:
metrics.opex("month-year")

OpEx
year month             
2003 1       331,672.34
     2       291,740.34
     3       330,141.09
     4       350,007.51
     5       341,141.09
...                 ...
2012 8       334,141.09
     9       418,007.51
     10      391,641.09
     11    4,229,052.65
     12    4,433,894.82

[120 rows x 1 columns]

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
